In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# Analyzes regression in app benchmark and prints relative differences in appRegression.csv

folder = "../../results_all/app_history_influxdb/"
filename = "../../results_aggr/app_history_influxdb.csv"
print(folder)

../../results_all/app_history_influxdb/


In [3]:
def reduce(row):
    row[2] = float(row[2].split("=")[1].split(")")[0])
    return row

rows = []
all_runs = glob.glob(folder + "*")
for fullRunFolder in all_runs:
    runFolder = fullRunFolder + "/"
    all_numbers = glob.glob(runFolder + "*")
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(fullNumberFolder + "/*")
        size = len(all_files)

        if size != 17:
            print(f"{size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split("\\")

            #Parse run and commit number from path structure
            run = tmp[len(tmp)-2]
            number = tmp[len(tmp)-1]
            print(f"Run {run} Number {number}")

            #Read latency file for both types (old and new)
            insertsOld = pd.read_csv(fullNumberFolder + "/latenciesInsertsOld.csv",
                             header=None,
                             names=["worker","batch","latency (ms)"])

            insertsNew = pd.read_csv(fullNumberFolder + "/latenciesInsertsNew.csv",
                             header=None,
                             names=["worker","batch","latency (ms)"])

            queries1Old = pd.read_csv(fullNumberFolder + "/latenciesQueries1Old.csv",
                             header=None,
                             names=["query","latency (ms)"])

            queries2Old = pd.read_csv(fullNumberFolder + "/latenciesQueries2Old.csv",
                             header=None,
                             names=["query","latency (ms)"])

            queries1New = pd.read_csv(fullNumberFolder + "/latenciesQueries1New.csv",
                             header=None,
                             names=["query","latency (ms)"])

            queries2New = pd.read_csv(fullNumberFolder + "/latenciesQueries2New.csv",
                             header=None,
                             names=["query","latency (ms)"])

            #Find median latency for both types
            medianInsertsOld = insertsOld["latency (ms)"].median()
            medianInsertsNew = insertsNew["latency (ms)"].median()

            medianQueries1Old = queries1Old["latency (ms)"].median()
            medianQueries1New = queries1New["latency (ms)"].median()

            medianQueries2Old = queries2Old["latency (ms)"].median()
            medianQueries2New = queries2New["latency (ms)"].median()

            #Calculate diff
            diffInserts = ((medianInsertsNew / medianInsertsOld) - 1) * 100
            diffQueries1 = ((medianQueries1New / medianQueries1Old) - 1) * 100
            diffQueries2 = ((medianQueries2New / medianQueries2Old) - 1) * 100

            #Append to rows
            rows.append({
                "run" : run,
                "number" : int(number),
                "diffInserts": diffInserts,
                "diffQueries1": diffQueries1,
                "diffQueries2": diffQueries2
            })

Run run1 Number 1
Run run1 Number 10
Run run1 Number 100
Run run1 Number 101
Run run1 Number 102
Run run1 Number 103
Run run1 Number 104
Run run1 Number 105
Run run1 Number 106
Run run1 Number 107
Run run1 Number 108
Run run1 Number 109
Run run1 Number 11
Run run1 Number 110
Run run1 Number 111
Run run1 Number 12
Run run1 Number 13
Run run1 Number 14
Run run1 Number 15
Run run1 Number 16
Run run1 Number 17
Run run1 Number 18
Run run1 Number 19
Run run1 Number 2
Run run1 Number 20
Run run1 Number 21
Run run1 Number 22
Run run1 Number 23
Run run1 Number 24
Run run1 Number 25
Run run1 Number 26
Run run1 Number 27
Run run1 Number 28
Run run1 Number 29
Run run1 Number 3
Run run1 Number 30
Run run1 Number 31
Run run1 Number 32
Run run1 Number 33
Run run1 Number 34
Run run1 Number 35
Run run1 Number 36
Run run1 Number 37
Run run1 Number 38
Run run1 Number 39
Run run1 Number 4
Run run1 Number 40
Run run1 Number 41
Run run1 Number 42
Run run1 Number 43
Run run1 Number 44
Run run1 Number 45
Run 

In [4]:
# Convert to data frame
df_result = pd.DataFrame(rows)
df_result.sort_values(by=["number"], inplace=True)
df_result.describe()


,number,diffInserts,diffQueries1,diffQueries2
count,314.000000,314.000000,314.000000,314.000000
mean,54.127389,4.940191,-6.167590,-0.986962
std,31.745841,4.370489,6.928249,4.786723
min,1.000000,-0.700920,-27.188988,-17.756120
25%,27.000000,0.132055,-7.182862,-0.446173
50%,53.000000,8.327257,-5.772010,0.344936
75%,79.000000,8.907322,-1.059058,1.121044
max,111.000000,9.906449,2.758067,2.949452


In [5]:
df_result.to_csv(filename)